In [1]:
from main import *
import transformers as ts
import torch

In [ ]:
class Dataset(torch.utils.data.Dataset):
        """
        Dataset object
        """

        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: val[idx].clone() for key, val in self.encodings.items()}
            item["labels"] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)
from datasets import load_metric
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base", cache_dir=None)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilroberta-base",
    cache_dir=None,
    num_labels=len(category_dict),
    problem_type="single_label_classification",
)

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="ex4\\transforers_results",  # output directory
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
)

In [9]:
category_dict = {
    "comp.graphics": "computer graphics",
    "rec.sport.baseball": "baseball",
    "sci.electronics": "science, electronics",
    "talk.politics.guns": "politics, guns",
}

In [16]:
x_train, y_train, x_test, y_test = get_data()

In [26]:
train = Dataset(tokenizer(x_train, return_tensors="pt"), y_train)
test = Dataset(tokenizer(x_test, return_tensors="pt"), y_test)

Token indices sequence length is longer than the specified maximum sequence length for this model (821 > 512). Running this sequence through the model will result in indexing errors


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)  # doctest: +SKIP

In [ ]:
trainer.train()

  0%|          | 0/2774 [00:00<?, ?it/s]

IndexError: Target 7 is out of bounds.